In [24]:
using Pkg; Pkg.activate("..")
using Base.Threads: @spawn
Threads.nthreads()

  Activating environment at `~/.julia/dev/PRONTO.jl/Project.toml`


1

In [25]:
using DifferentialEquations
using LinearAlgebra
using Colors
# using ColorSchemes
set_alpha(CLR::T, alpha) where {T} = coloralpha(T)(CLR.r, CLR.g, CLR.b, alpha)

set_alpha (generic function with 1 method)

In [35]:
# interactive, standalone window
using GLMakie
GLMakie.activate!()

In [27]:
# inline plots
using CairoMakie
CairoMakie.activate!(type = "png")
CairoMakie.activate!(type = "svg")

In [37]:
clr_bg = parse(RGBAf, "#272822") # background
clr_mg = parse(RGBAf, "#8E8A73") # mid-ground
clr_fg = :white # fg

clr_set = [
    parse(RGBAf, "#0072BD"), # blue
    parse(RGBAf, "#D95319"), # orange/red
    parse(RGBAf, "#EDB120"), # yellow
    parse(RGBAf, "#7E2F8E"), # purple
    parse(RGBAf, "#009E73FF"), # teal
]

set_theme!(Theme(
    resolution = (1500, 1000),
    textcolor = :white,
    linewidth = 3,
    fontsize = 16,
    font = "Inter",
    color = clr_fg,
    palette = (
        color = clr_set,
    ),
    markercolor = :white,
    backgroundcolor = clr_bg,
    Axis = (
        backgroundcolor = clr_bg,
        xgridcolor = clr_mg,
        ygridcolor = clr_mg, 
        topspinecolor = clr_mg, 
        rightspinecolor = clr_mg,
        leftspinecolor = clr_mg, 
        bottomspinecolor = clr_mg,
        xtickcolor = clr_mg,
        ytickcolor = clr_mg,
    ),
))


In [39]:
## -------------------- plotting helper functions --------------------- ##

function phi2xy(ϕ, i)
    x = -l*sum(sin.(ϕ[1:i]))
    y = l*sum(cos.(ϕ[1:i]))
    return x, y
end

function phis2points(ϕvec)
    return [Point2f(phi2xy(ϕvec, i)) for i=1:N]
end

function colortomap(color, len)
    # Colors.lsequential_palette(color.h, )
    cmap = range(RGB(1.,1.,1.), stop=color, length=len)
    return cmap
end

function kinetic(x)
    ϕ = x[1:N]
    ϕd = x[N+1:end]
    T = 0
    for i = 1:N
        T += 1/2 * l^2 * m[i] * sum(sum([ϕd[j] * ϕd[k] * cos(ϕ[j]-ϕ[k]) for j = 1:i, k=1:i]))
    end
    return T
end

function potential(x)
    ϕ = x[1:N]
    V = 0
    for i = 1:N
        V += m[i] * g * l * sum([cos(ϕ[j]) for j = 1:i])
    end
    return V
end

function ϕdϕ(x)
    n = Int(length(x)/2)
    return (x[1:n], x[n+1:end])
end

function getϕ(x)
    n = Int(length(x)/2)
    return x[1:n]
end

getϕ (generic function with 1 method)

In [41]:
# plotting code
function make_plots(x)
    t = x.t[1]:0.01:x.t[end]
    ϕt = [map(tx->x(tx)[ix], t) for ix in 1:N]
    dϕt = [map(tx->x(tx)[ix], t) for ix in N+1:2N]

    fig = Figure()
    axs = Axis[]

    ax = Axis(fig[1,1]; title="ϕ(t)")
    push!(axs, ax)
    for ϕ in ϕt
        lines!(ax, t, ϕ)
    end

    ax = Axis(fig[2,1]; title="dϕ(t)")
    push!(axs, ax)
    for dϕ in dϕt
        lines!(ax, t, dϕ)
    end

    ax = Axis(fig[1,2]; title="G(t)")
    push!(axs, ax)
    for ix in 1:N
        lines!(ax, t, map(tx->G(x(tx)[1:N])[ix], t))
    end

    # map(tx->C(ϕdϕ(x(tx))...), t)
    ax = Axis(fig[2,2]; title="C(t)")
    push!(axs, ax)
    for ix in 1:N
        lines!(ax, t, map(tx->C(ϕdϕ(x(tx))...)[ix], t))
    end

    # @. potential(getϕ(x(t)))
    ax = Axis(fig[1:2,3]; title="energy")
    push!(axs, ax)
    lines!(ax, t, @. potential(x(t)))
    lines!(ax, t, @. kinetic(x(t)))
    lines!(ax, t, @. potential(x(t)) + kinetic(x(t)))
    
    display(fig)
    return (fig, axs)
end

make_plots (generic function with 1 method)

In [113]:
## ---------------------------- build ODE ------------------------------ ##

# parameters
g = 9.8
l = 1
N = 4
# m = [1, 20, 1, 1]
m = ones(N)
T = t -> zeros(N)

tspan = (0.0, 40.0)
θ₀ = zeros(N)
θ₀[N] = π/2
θd₀ = zeros(N)

# dynamics
# mass matrix M:
U = UpperTriangular(ones(N,N))
L = LowerTriangular(ones(N,N))
Linv = inv(L)
v1 = ones(N)

# ℳ =  (U * m) .* I(N)
ℳvec = (U * m) 
ℳ = [ℳvec[ max(i,j)] for i in 1:N, j in 1:N]
𝒞 = ϕ -> [cos(ϕ[i] - ϕ[j]) for i in 1:N, j in 1:N]
M = ϕ -> l^2 .* ℳ .* 𝒞(ϕ)
# M = ϕ -> [  sum(
#                 [  [ l^2 * m[i] * cos(ϕ[a]-ϕ[j]) for j in 1:i ]
#                 for i in a:N]
#                 ) 
#             for a in 1:N]

# coriolis vector C:
# 𝒮 = ϕ -> [sin(ϕ[i] - ϕ[j]) for i in 1:N, j in 1:N]
# C = (ϕ, ϕd) -> l^2 .* ℳ .* 𝒮(ϕ) .* (v1*ϕd' - 2ϕd*v1') * ϕd
# C = (ϕ, ϕd) -> [ sum(
#                     [ sum(
#                         [ -l^2 * m[i] * sin(ϕ[a]-ϕ[j]) * (ϕd[j]^2 + 2*ϕd[a]*ϕd[j]) 
#                         for j in 1:i ]) 
#                     for i in a:N]) 
#                 for a in 1:N]
# C = (ϕ, ϕd) -> 1^2 * sum([m[i] * 𝒮[i,j]])
N=2
C = (ϕ, ϕd) ->[ sum(
                    [ sum(
                        [ l^2 * m[i] * sin(ϕ[a]-ϕ[j]) * ϕd[j]^2 
                        for j in 1:i ]) 
                    for i in a:N]) 
                for a in 1:N]

# body force vector G:
G = ϕ -> @. -g*l*ℳvec*sin(ϕ)


# ODE solver formulation:
function f!(dx, x, T, t)
    # L: θ -> ϕ, Linv: ϕ -> θ
    θ = x[1:N]; θd = x[N+1:end]
    ϕ = L * θ
    ϕd = L * θd
    θdd = Linv*inv(M(ϕ)) * (-C(ϕ, ϕd) - G(ϕ) + Linv*T(t))
    dx[1:N] = θd; dx[N+1:end] = θdd
end

function fϕ!(dx, x, T, t)
    # x = [ϕ,dϕ], dx = [dϕ,ddϕ]
    ϕ = x[1:N]
    dϕ = dx[1:N] .= x[N+1:end]
    dx[N+1:end] .= inv(M(ϕ)) * (-C(ϕ, dϕ) - G(ϕ) + Linv*T(t))
end

C([0, pi/2], [1, 1])

2-element Vector{Float64}:
 -1.0
  1.0

In [109]:
## ---------------------------- solve ODE ------------------------------ ##

# x = solve(ODEProblem(fϕ!, [θ₀; θd₀], tspan, T));
x = solve(ODEProblem(fϕ!, [θ₀; θd₀], tspan, T), Rosenbrock23()) # matlab ode23s
# x = solve(ODEProblem(fϕ!, [θ₀; θd₀], tspan, T), TRBDF2())
# x = solve(ODEProblem(fϕ!, [θ₀; θd₀], tspan, T), BS3()) ;# matlab ode23


In [110]:
make_plots(x);

In [69]:
tp = 2.85:.01:2.95
tp = 2.7:.01:2.9
@show ϕ = [ui[1:2] for ui in x(tp).u]
@show dϕ = [ui[3:4] for ui in x(tp).u]
Ctp = C.(ϕ, dϕ)

ϕ = [ui[1:2] for ui = (x(tp)).u] = [[15.879545853813015, 18.33072044265601], [15.943159926102004, 18.342447229244897], [16.0047727725859, 18.350439331246026], [16.0644868373616, 18.35452975783393], [16.122442949269214, 18.354546047373383], [16.178820321892037, 18.35031026741939], [16.233836553556575, 18.341639014717202], [16.28774762733251, 18.328343415202323], [16.34088963864876, 18.310155264090348], [16.393643894381963, 18.28672394683267], [16.446469013597813, 18.257593401007245], [16.50005293927264, 18.221990126052134], [16.555314895528475, 18.178820227189693], [16.613478939550394, 18.12656257771878], [16.67631959478038, 18.062863553405563], [16.74697637947467, 17.983348359735015], [16.83168314801449, 17.879196542040482], [16.943694536502846, 17.731434281059016], [17.122224913974943, 17.483818603419113], [17.397600348877038, 17.094102531647305], [17.58502797071922, 16.830468277381073]]
dϕ = [ui[3:4] for ui = (x(tp)).u] = 

[[6.465976767216722, 1.3482922491330427], [6.261874031402921, 0.9806970767426789], [6.065499182346242, 0.5991803255122974], [5.8808414446216, 0.20303291135654533], [5.712052360071486, -0.20913881486137756], [5.56344578780604, -0.6394130673295427], [5.43949790420298, -1.0905526252874613], [5.344847202907672, -1.5660048330258913], [5.286619033949523, -2.0736279364426986], [5.272266181073197, -2.6230683417930547], [5.311382485454047, -3.227332952160286], [5.4243188520700265, -3.9154323170770975], [5.64229371269233, -4.732548481498853], [6.015691362877854, -5.751980162922124], [6.619751621967826, -7.084178343034828], [7.621025472095183, -8.972671989900247], [9.494520629646818, -12.105155242754357], [13.482467956297883, -18.272957846206054], [23.783268137373092, -33.472879413737864], [25.136872720896225, -35.576579317767354], [13.889042555406222, -19.348872551708308]]


21-element Vector{Vector{Float64}}:
 [9.94657492766688, 15.522052426658298]
 [7.652350129459034, 18.20373835590236]
 [4.9370184319346, 21.09355011486956]
 [1.7654927742137656, 24.221341055221426]
 [-1.920070745759496, 27.63489385512405]
 [-6.2064883283895576, 31.402811854737713]
 [-11.216076099651618, 35.617604998502635]
 [-17.113153063030875, 40.39884850602594]
 [-24.17033145337118, 45.966028519583034]
 [-32.7612343254992, 52.60065240548631]
 ⋮
 [-75.69659872419788, 85.12195381574931]
 [-102.11924715672838, 105.21734726001503]
 [-141.53956531182817, 135.28282227399228]
 [-205.23383763158918, 184.04759372874193]
 [-326.03145300691415, 277.18836391540816]
 [-585.8828548268989, 478.0633261924225]
 [-959.6317824614812, 763.3663899832777]
 [912.7963616549885, -723.3697506773559]
 [624.5904112845055, -500.2869737718404]